# get_frequently_mutated Final 

In [4]:
import cptac
#en = cptac.Endometrial()
#cptac.sync('endometrial', version='2.1')
#ov = cptac.Ovarian()
#cptac.sync(dataset='ovarian', version='0.0')
#cptac.sync(dataset='colon', version='0.0')
#ov = cptac.Ovarian()

import pandas as pd
import numpy as np

In [32]:
def get_frequently_mutated(cancer_object, cutoff = 0.1):  
    """
    take cancer object and find the frequently 
    mutated genes in the total tumors compared to the cutoff.

    Parameters:
    cancer_object (object): cancer class object from cptac module 
    cutoff (float): used as comparison to determine the 
                    status of gene mutation frequency

    Returns:
    freq_mutated_df (pd.DataFrame): DataFrame of frequently 
        mutated genes passing the cutoff. Columns contain the 
        fractions of total unique mutations,missence type 
        mutations, and truncation type mutations per gene.
    
    The Missence_Mut column includes: 
        In_Frame_Del, In_Frame_Ins, Missense_Mutation
   
   The Truncation_Mut column includes: 
        Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, 
        Nonsense_Mutation, Nonstop_Mutation
        
    These columns count multiple mutations of one gene in the 
    same sample, so fractions in the last two columns may 
    exceed the Unique_Samples_Mut column which only counts if 
    the gene was mutated once per sample."""    
    
    # Get mutations data frame (whole exome sequencing)
    somatic_mutations = cancer_object.get_mutations()
    somatic_mutations = somatic_mutations.loc[somatic_mutations['Gene'] == 'CASP5']
             
    # Get total tumors/patients
    omics_and_mutations = cancer_object.append_mutations_to_omics(
        mutation_genes = 'TP53', omics_df_name = 'proteomics', omics_genes = 'TP53')
    tumors = omics_and_mutations.loc[omics_and_mutations['Sample_Status'] == 'Tumor']
    total_tumor_patients = len(tumors)
    #print(total_tumor_patients)
    
    # Find frequently mutated genes and their fraction of unique mutated samples.
    #move 'Sample_ID' from index to column
    origin_df = somatic_mutations.reset_index()
    
    # Drop silent mutations for Ovarian dataset
    if cancer_object.get_cancer_type() == 'ovarian':
        origin_df = origin_df.loc[origin_df['Mutation'] != 'Silent']
        #print(origin_df['Mutation'].unique())
        
    #group by gene and count unique samples
    count_mutations = origin_df.groupby(['Gene']).nunique()

    #format
    count_mutations = count_mutations.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
    count_mutations = count_mutations.drop(['Gene', 'Mutation','Location'], axis = 1)
    
    #filter using the cutoff and create fraction
    fraction_mutated = count_mutations.apply(lambda x: x / total_tumor_patients)
    fraction_greater_than_cutoff = fraction_mutated.where(lambda x: x > cutoff) #na used when false
    filtered_gene_df = fraction_greater_than_cutoff.dropna()
    
    
    #Create Missence and Trucation data frame
    
    #create two categories in Mutation column
    if cancer_object.get_cancer_type() == 'colon':
        missence_truncation_groups = {'frameshift substitution': 'T', 
            'frameshift deletion': 'T', 'frameshift insertion': 'T', 
            'stopgain': 'T', 'stoploss': 'T', 'nonsynonymous SNV': 'M',
            'nonframeshift insertion': 'M','nonframeshift deletion': 'M', 
            'nonframeshift substitution': 'M'}
    else: 
        missence_truncation_groups = {'In_Frame_Del': 'M', 'In_Frame_Ins': 'M',
            'Missense_Mutation': 'M', 'Frame_Shift_Del': 'T','Nonsense_Mutation': 'T', 
            'Splice_Site': 'T', 'Frame_Shift_Ins': 'T','Nonstop_Mutation':'T'}
       
    mutations_replaced_M_T = origin_df.replace(missence_truncation_groups)
    ##print(mutations_replaced_M_T)
    # group mutation categories
    miss = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'M']
    trunc = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'T']
    
    # group by gene and count unique samples for both categories
    count_miss = miss.groupby(['Gene']).nunique()
    count_trunc = trunc.groupby(['Gene']).nunique()
    print(count_miss)
    if count_miss.empty():
        print('j')
    #format
    missence_df = count_miss.rename(columns={"Sample_ID": "Missence_Mut"})
    missence_df = missence_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)

    truncation_df = count_trunc.rename(columns={"Sample_ID": "Truncation_Mut"})
    truncation_df = truncation_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)
    
    
    
    #join miss and trunc and change nan to 0, then divide by total tumors
    join_mutations = missence_df.join(truncation_df).fillna(0)
    missence_and_truncation_df = join_mutations.apply(lambda x: x / total_tumor_patients)
    #print(join_mutations)

    #Join data frames, keeping only the genes that passed the cutoff 
    freq_mutated_df = filtered_gene_df.join(missence_and_truncation_df).reset_index()
    freq_mutated_df.name = 'frequently_mutated'
    
    return freq_mutated_df

In [28]:
df = get_frequently_mutated(ov, .1)

NameError: name 'ov' is not defined

In [25]:
df

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,BIRC6,0.120482,0.108434,0.024096
1,FSIP2,0.108434,0.096386,0.024096
2,MT-CO1,0.120482,0.120482,0.000000
3,MT-CO3,0.120482,0.108434,0.012048
4,MT-ND5,0.108434,0.084337,0.024096
5,MUC12,0.144578,0.144578,0.012048
6,MUC16,0.144578,0.144578,0.012048
7,MUC17,0.144578,0.144578,0.000000
8,MUC4,0.325301,0.313253,0.012048
9,MUC5B,0.108434,0.108434,0.000000


In [6]:
co = cptac.Colon()

In [33]:
colon_freq_mutated_df = get_frequently_mutated(co, .15)

Empty DataFrame
Columns: [Sample_ID, Gene, Mutation, Location]
Index: []


TypeError: 'bool' object is not callable

In [24]:
colon_freq_mutated_df

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,CASP5,0.195876,NaN,NaN


ValueError: too many values to unpack (expected 1)

In [6]:
#test results
m = co.get_mutations()
gene = 'CASP5'
g = m.loc[m['Gene'] == gene]
print(g)
print(len(g.index.unique()))

r = g.groupby('Mutation')

r.groups


            Gene                 Mutation    Location
Sample_ID                                            
S006       CASP5  frameshift substitution     K9Nfs*2
S006       CASP5  frameshift substitution    K80Nfs*2
S006       CASP5  frameshift substitution    K67Nfs*2
S009       CASP5  frameshift substitution    K67Nfs*2
S009       CASP5  frameshift substitution   T68Qfs*25
S009       CASP5  frameshift substitution   T10Qfs*25
S009       CASP5  frameshift substitution   T81Qfs*25
S009       CASP5  frameshift substitution    K80Nfs*2
S009       CASP5  frameshift substitution     K9Nfs*2
S014       CASP5  frameshift substitution   T10Qfs*25
S014       CASP5  frameshift substitution   T68Qfs*25
S014       CASP5  frameshift substitution   T81Qfs*25
S018       CASP5  frameshift substitution    K67Nfs*2
S018       CASP5  frameshift substitution   T68Qfs*25
S018       CASP5  frameshift substitution   T81Qfs*25
S018       CASP5  frameshift substitution   T10Qfs*25
S018       CASP5  frameshift

{'frameshift deletion': Index(['S055', 'S055', 'S055', 'S055', 'S074', 'S074', 'S074', 'S080', 'S080',
        'S080', 'S081', 'S081'],
       dtype='object', name='Sample_ID'),
 'frameshift insertion': Index(['S031', 'S031', 'S031', 'S031'], dtype='object', name='Sample_ID'),
 'frameshift substitution': Index(['S006', 'S006', 'S006', 'S009', 'S009', 'S009', 'S009', 'S009', 'S009',
        'S014', 'S014', 'S014', 'S018', 'S018', 'S018', 'S018', 'S018', 'S018',
        'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S023', 'S028',
        'S028', 'S028', 'S028', 'S030', 'S030', 'S030', 'S033', 'S033', 'S033',
        'S033', 'S035', 'S035', 'S035', 'S035', 'S035', 'S052', 'S052', 'S052',
        'S052', 'S052', 'S052', 'S052', 'S052', 'S052', 'S052', 'S055', 'S055',
        'S055', 'S055', 'S064', 'S064', 'S064', 'S065', 'S065', 'S065', 'S065',
        'S065', 'S065', 'S065', 'S065', 'S074', 'S074', 'S074', 'S080', 'S080',
        'S080', 'S101', 'S101', 'S101', 'S108', 'S108',

In [19]:
en_freq_mutated_df = get_frequently_mutated(en,.1)

95


In [ ]:
ovarian_freq_mutated_df